In [2]:
import numba
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from numba import njit
import scipy
from scipy.spatial.transform import Rotation as Rot


In [54]:
allPossbileActions = np.load("allPossbileActions.npy")
actionsData = np.load("actionDatasForE=3.npy")
states = np.zeros((10,4),dtype=np.int64)

In [4]:

@njit
def getAction(state,ratio):
    #t=1 if state=0 and t = ratio if state = 1   ???
    t1,t2,t3,t4 = np.where(state==0,1,ratio)
    sum = t1*t2*t3 + t1*t2*t4 + t1*t3*t4 + t2*t3*t4
    randomNumber = np.random.uniform(0,1,1)
    if randomNumber <= t4*t2*t3/sum:
        changingJoint = 0
        tc= t1
    elif(randomNumber <= (t4*t3*t1+t4*t2*t3)/sum):  
        changingJoint = 1
        tc=t2
    elif randomNumber <= (t4*t2*t1+t4*t2*t3+t4*t3*t1)/sum:
        changingJoint = 2
        tc=t3
    else:
        changingJoint = 3
        tc=t4

    # deltat = getDeltat2(t1,t2,t3,t4)   ### check detla t
    # print("t= ",getDeltat2(t1,t2,t3,t4) )
    #deltat = np.array([t1,t2,t3,t4])[changingJoint]
    #print(getDeltat(t1,t2,t3,t4,tc))
    deltat = getDeltat(t1,t2,t3,t4,tc)
    return changingJoint,deltat



In [18]:
getAction(states[0],1)

(2, 1.17775)

In [114]:

@njit
def doStep(Data,state,ratio):
    action,deltat = getAction(state,ratio)
    new_state = state.copy()
    new_state[action] = 1 - new_state[action]
    delta = Data[fromBinary(state)*4 + action][0,:]    ### check data
    rotation = Data[fromBinary(state)*4 + action][1:,:]
    print(action,new_state)
    return new_state,delta,rotation,deltat


In [132]:
state = np.zeros(4,dtype=np.int64)
doStep(actionsData,state,1)

0 [1 0 0 0]


(array([1, 0, 0, 0]),
 array([-3.43239200e-18,  7.11006105e-18, -5.41599075e-01]),
 array([[ 1.00000000e+00, -1.95126984e-17,  3.21804178e-19],
        [-1.26908362e-18,  1.00000000e+00, -4.68240650e-19],
        [-1.11022302e-16, -1.44291227e-16,  1.00000000e+00]]),
 1.17775)

In [65]:
perturbingSteps = 10
state = np.zeros(4)
time = 0
ratio =1
pertrubingPose = np.array([0.,0.,0.])
perturbingRotation = np.eye(3)
state = np.ones(4,dtype=np.int64)
pertrubingPoses = np.zeros((perturbingSteps,3))
pertrubingTimes = np.zeros(perturbingSteps)

In [62]:
state

array([1, 1, 1, 1])

In [60]:
doStep(actionsData,state,ratio)

(array([0, 1, 1, 1]),
 array([ 9.23204540e-18, -5.41258007e-18,  5.57630488e-01]),
 array([[ 1.00000000e+00, -6.94160846e-17, -3.38010295e-18],
        [-6.67015326e-19,  1.00000000e+00, -2.95931133e-19],
        [ 8.32667268e-16,  1.43062525e-15,  1.00000000e+00]]),
 1.17775)

In [49]:

for t in range(perturbingSteps):
    newState,delta,rotation,deltat = doStep(actionsData,state,ratio)
    state = newState
    pertrubingPose += perturbingRotation @ delta
    perturbingRotation = rotation @ perturbingRotation
    pertrubingPoses[t] = pertrubingPose
    time += deltat
    pertrubingTimes[t] = time

In [7]:

@njit
def getDeltat(t1,t2,t3,t4,tc):
    tav = 18.844
    return (tav/tc)/(((1/t1 + 1/t2 + 1/t3 + 1/t4)**2))


In [8]:


@njit
def getDeltat2(t1,t2,t3,t4):
    return 1/(1/t1 + 1/t2 + 1/t3 + 1/t4)



In [9]:

@njit
def getBinary(a):
    return np.array([a//8,(a%8)//4,(a%4)//2,a%2])



In [10]:

@njit 
def fromBinary(binary):
    x = 0
    for i in range(len(binary)):
        x += binary[::-1][i]*(2**i)
    return x


In [11]:




@njit
def perturbe(Data,perturbingSteps,ratio):
    time = 0
    pertrubingPose = np.array([0.,0.,0.])
    perturbingRotation = np.eye(3)
    state = np.ones(4,dtype=np.int64)
    pertrubingPoses = np.zeros((perturbingSteps,3))
    pertrubingTimes = np.zeros(perturbingSteps)
        
    for t in range(perturbingSteps):
        newState,delta,rotation,deltat = doStep(Data,state,ratio)
        state = newState
        pertrubingPose += perturbingRotation @ delta
        perturbingRotation = rotation @ perturbingRotation
        pertrubingPoses[t] = pertrubingPose
        time += deltat
        pertrubingTimes[t] = time
        
    return pertrubingPoses,pertrubingTimes


In [ ]:


# @njit
# def getPerturbingDiffusionCoefForRatio(Data,ratio,iterations=1000,perturbingSteps=1000):
#     # iterations = 10_00
#     # perturbingSteps = 10_00
#     meanFinalPoses = np.zeros((perturbingSteps))
#     meanFinalTimes = np.zeros(perturbingSteps)
#     for iteration in range(iterations):
#         pertrubingPoses,pertrubingTimes = perturbe(Data,perturbingSteps,ratio)
#         #print(np.sum(np.square(pertrubingPoses[:]),axis=1).shape,meanFinalPoses.shape)
#         meanFinalPoses +=   np.sum(np.square(pertrubingPoses[:]),axis=1)
#         meanFinalTimes += pertrubingTimes
#         #print(iteration)
     
#     return meanFinalTimes/iterations,meanFinalPoses/iterations

# def getPerturbingDiffusionCoef(Data,ratios,iterations=1000,perturbingSteps=1000):
#     diffs =np.zeros(ratios.shape)
#     for i in range(ratios.shape[0]):
#         print(i)
#         meanFinalTimes,meanFinalPoses= getPerturbingDiffusionCoefForRatio(Data,ratios[i],iterations,perturbingSteps)
#         slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(meanFinalPoses,meanFinalTimes)
#         diffs[i] =  slope 
#     return diffs



In [ ]:


def getPerturbingDiffusionCoefForRatio(Data,ratio,iterations=1000,perturbingSteps=1000):
    iterationDiffs = np.zeros(iterations)
    for iteration in range(iterations):
        pertrubingPoses,pertrubingTimes = perturbe(Data,perturbingSteps,ratio)


        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(pertrubingTimes,np.sum(np.square(pertrubingPoses),axis=1))
        iterationDiffs[iteration] =  slope 

     
    return np.mean(iterationDiffs)


In [ ]:

def getPerturbingDiffusionCoef(Data,ratios,iterations=1000,perturbingSteps=1000):
    diffs =np.zeros(ratios.shape)
    for i in range(ratios.shape[0]):
        print(i)
        diffs[i] = getPerturbingDiffusionCoefForRatio(Data,ratios[i],iterations,perturbingSteps)
    return diffs



In [ ]:

ratios = np.exp(np.linspace(-5,5,50))


In [ ]:


diffs = getPerturbingDiffusionCoef(actionsData,ratios,100,10000)


In [ ]:

np.save("diffs1",diffs)
plt.plot(np.log(ratios),diffs)
plt.savefig("plot.png")
plt.show()